In [ ]:
from moviepy import video
from moviepy.editor import *
from multiprocessing import Process, Semaphore
import sys
import soundfile as sf
import pyloudnorm as pyln
import pandas as pd
import os
from moviepy.editor import VideoFileClip, concatenate_videoclips

def break_files(video_path,csv_path,temp_output_folder):
     df = pd.read_csv(csv_path)
     num = 0
     for ind in df.index:
          #print(df['start time'][ind], df['end time'][ind])
          with VideoFileClip(video_path) as clip:
               clip1 = clip.subclip(df['start time'][ind], df['end time'][ind])
               clip1.write_videofile(os.path.join(temp_output_folder,"output_%s.mp4" % num))
               clip1.audio.write_audiofile(os.path.join(temp_output_folder,"output_%s.wav" % num))
               num += 1

def get_loudness(directory):
     dict1={}
     for filename in os.listdir(directory):
         if filename.endswith(".wav"):
              path = os.path.join(directory,filename)
              data, rate = sf.read(path)  # load audio (with shape (samples, channels))
              meter = pyln.Meter(rate)  # create BS.1770 meter
              loudness = meter.integrated_loudness(data)  # measure loudness
              dict_key=filename.split(".")[0] + ".mp4"
              dict1[dict_key] = loudness
              #print(loudness)
     return dict1

def sort_dict(dict1,directory):
     list1 = sorted(dict1.items(), key=lambda kv: (kv[1], kv[0]), reverse=True)
     list1=list1[:3]
     list2=[]
     for i in list1:
          list2.append(i[0])
     list2=sorted(list2)
     clip1=[]
     for i in list2:
          clip1.append(VideoFileClip(os.path.join(directory,i)))
     final_clip = concatenate_videoclips(clip1)
     final_clip.write_videofile(os.path.join(directory,"my_concatenation.mp4"))
     return list1

if __name__ == "__main__":
     file_path = input("Enter video path ")
     excel_path = input("Enter excel path ")
     final_output_path = input("Enter final path to store output video ")

     if os.path.exists(file_path) and os.path.exists(excel_path) and os.path.exists(final_output_path):
          print("abc")
          temp_folder = r"C:\Users\ADMIN\Desktop\tennispath\broken_files"
          break_files(file_path,excel_path,temp_folder)
          dict1 = get_loudness((temp_folder))
          sort_dict(dict1,temp_folder)
     else:
          print("Your input is invalid")

# break_files()
# directory = r"C:\Users\ADMIN\Desktop\tennispath\broken_files"
# dict1 = {}
# try:
#      print("new resources open")
#
#      for filename in os.listdir(directory):
#          if filename.endswith(".wav"):
#               path = os.path.join(directory,filename)
#               data, rate = sf.read(path)  # load audio (with shape (samples, channels))
#               meter = pyln.Meter(rate)  # create BS.1770 meter
#               loudness = meter.integrated_loudness(data)  # measure loudness
#               dict_key=filename.split(".")[0] + ".mp4"
#               dict1[dict_key] = loudness
#               #print(loudness)
#      #print(dict1)
#      list1 = sorted(dict1.items(), key=lambda kv: (kv[1], kv[0]),reverse=True)
#      list1=list1[:3]
#      print(list1)
#
#
#      list2=[]
#      for i in list1:
#           list2.append(i[0])
#      list2=sorted(list2)
#      print(list2)
#      clip1=[]
# except KeyError as err:
#      print("value in a dict using a invalid key",err)
#
#
# for i in list2:
#      clip1.append(VideoFileClip(os.path.join(directory,i)))
#
# final_clip = concatenate_videoclips(clip1)
# final_clip.write_videofile(os.path.join(directory,"my_concatenation.mp4"))
